In [ ]:
import json

class Policyholder:
    def __init__(self, name, id_number):
        self.name = name
        self.id_number = id_number
        self.active = True
        self.policies = []
        self.payments = []

    def suspend(self):
        self.active = False

    def reactivate(self):
        self.active = True

    def add_policy(self, product):
        self.policies.append(product)

    def add_payment(self, payment):
        self.payments.append(payment)

    def to_dict(self):
        return {
            "name": self.name,
            "id_number": self.id_number,
            "active": self.active,
            "policies": [p.product_id for p in self.policies],
            "payments": [p.status for p in self.payments]
        }

    @classmethod
    def from_dict(cls, data):
        holder = cls(data["name"], data["id_number"])
        holder.active = data["active"]
        holder.policies = data["policies"]  # Simplified for now, might need to match product IDs
        holder.payments = data["payments"]
        return holder

    def __str__(self):
        # Custom string representation for printing
        return f"Policyholder {self.name} (ID: {self.id_number}), Active: {self.active}"



class Product:
    def __init__(self, name, product_id, cost):
        self.name = name
        self.product_id = product_id
        self.cost = cost
        self.active = True

    def suspend(self):
        self.active = False

    def update(self, name=None, cost=None):
        if name: self.name = name
        if cost: self.cost = cost

    def to_dict(self):
        return {
            "name": self.name,
            "product_id": self.product_id,
            "cost": self.cost,
            "active": self.active
        }

    @classmethod
    def from_dict(cls, data):
        return cls(data["name"], data["product_id"], data["cost"])

class Payment:
    def __init__(self, policyholder, product):
        self.policyholder = policyholder
        self.product = product
        self.status = "Pending"

    def process(self):
        self.status = "Paid"

    def to_dict(self):
        return {
            "policyholder_id": self.policyholder.id_number,
            "product_id": self.product.product_id,
            "status": self.status
        }

    @classmethod
    def from_dict(cls, data, policyholders, products):
        policyholder = policyholders.get(data["policyholder_id"])
        product = products.get(data["product_id"])
        payment = cls(policyholder, product)
        payment.status = data["status"]
        return payment


def save_data():
    # Save policyholders
    with open("policyholders.json", "w") as f:
        json.dump({id_num: ph.to_dict() for id_num, ph in policyholders.items()}, f)

    # Save products
    with open("products.json", "w") as f:
        json.dump({pid: prod.to_dict() for pid, prod in products.items()}, f)

    # Save payments
    with open("payments.json", "w") as f:
        json.dump([payment.to_dict() for payment in payments], f)


def load_data():
    global policyholders, products, payments

    # Load policyholders
    try:
        with open("policyholders.json", "r") as f:
            data = json.load(f)
            policyholders = {k: Policyholder.from_dict(v) for k, v in data.items()}
    except FileNotFoundError:
        policyholders = {}

    # Load products
    try:
        with open("products.json", "r") as f:
            data = json.load(f)
            products = {k: Product.from_dict(v) for k, v in data.items()}
    except FileNotFoundError:
        products = {}

    # Load payments
    try:
        with open("payments.json", "r") as f:
            data = json.load(f)
            payments = [Payment.from_dict(p, policyholders, products) for p in data]
    except FileNotFoundError:
        payments = []


# --- Main Logic with Input ---
policyholders = {}
products = {}
payments = []

# Load data on program start
load_data()

def register_policyholder():
    name = input("Enter policyholder name: ")
    id_num = input("Enter ID number: ")
    holder = Policyholder(name, id_num)
    policyholders[id_num] = holder
    save_data()  # Save after each change
    print("Registered successfully.\n")

def create_product():
    name = input("Enter product name: ")
    product_id = input("Enter product ID: ")
    cost = float(input("Enter product cost: "))
    products[product_id] = Product(name, product_id, cost)
    save_data()  # Save after each change
    print("Product created.\n")

def process_payment():
    id_num = input("Enter policyholder ID: ")
    if id_num not in policyholders:
        print("Policyholder not found.")
        return
    print("Available products:")
    for pid, prod in products.items():
        print(f"{pid}: {prod.name} (${prod.cost})")
    selected = input("Enter product ID to pay for: ")
    if selected not in products:
        print("Invalid product.")
        return
    holder = policyholders[id_num]
    product = products[selected]
    holder.add_policy(product)
    payment = Payment(holder, product)
    payment.process()
    payments.append(payment)
    holder.add_payment(payment)
    save_data()  # Save after each change
    print("Payment processed.\n")

def suspend_reactivate_holder():
    id_num = input("Enter policyholder ID: ")
    if id_num in policyholders:
        action = input("Type 'suspend' or 'reactivate': ").lower()
        if action == 'suspend':
            policyholders[id_num].suspend()
            save_data()  # Save after each change
            print("Policyholder suspended.\n")
        elif action == 'reactivate':
            policyholders[id_num].reactivate()
            save_data()  # Save after each change
            print("Policyholder reactivated.\n")
        else:
            print("Invalid action.\n")
    else:
        print("Policyholder not found.\n")

def view_policyholder():
    id_num = input("Enter policyholder ID: ")
    if id_num in policyholders:
        print(policyholders[id_num])
    else:
        print("Policyholder not found.\n")

# --- Menu ---
def main():
    while True:
        print("\n--- Policy Management Menu ---")
        print("1. Register Policyholder")
        print("2. Create Product")
        print("3. Process Payment")
        print("4. Suspend/Reactivate Policyholder")
        print("5. View Policyholder Details")
        print("6. Exit")

        choice = input("Choose an option: ")

        if choice == "1":
            register_policyholder()
        elif choice == "2":
            create_product()
        elif choice == "3":
            process_payment()
        elif choice == "4":
            suspend_reactivate_holder()
        elif choice == "5":
            view_policyholder()
        elif choice == "6":
            print("Exiting system.")
            break
        else:
            print("Invalid choice.\n")

main()



--- Policy Management Menu ---
1. Register Policyholder
2. Create Product
3. Process Payment
4. Suspend/Reactivate Policyholder
5. View Policyholder Details
6. Exit


Choose an option:  5
Enter policyholder ID:  12


Policyholder kk (ID: 12), Active: True

--- Policy Management Menu ---
1. Register Policyholder
2. Create Product
3. Process Payment
4. Suspend/Reactivate Policyholder
5. View Policyholder Details
6. Exit
